# PyCity Schools Analysis

-To receive all points, the written report presents a cohesive written analysis that:

-Summarizes the analysis (5 points)

-Draws two correct conclusions or comparisons from the calculations (10 points)

Summary: We collected information about students in our local school district for analysis. There is a fairly large difference in the percentage of students passing both math and reading at the various high schools - the lowest is Rodriguez High School at 52.988247 percent, while the highest was Cabrera High School with 91.334769 percent. This means that achievement depends heavily on school name. From looking at the table of highest-performing and lowest-performing schools, it seems like there is a group of schools in the 90's as well as the 50's. As a result, we may label some schools high-performing and some schools low-performing. Further analysis on this disparity is necessary.

Our first goal is to see if there is a correlation between student success and funding. For this, we can use the per_school_summary dataframe to see if there is a relationship. For each school, the percent passing seems to decrease with an increase in spending per student. Looking at the spending_summary dataframe, we can see the same pattern. This is counterintuitive, so more investigation is needed.

Our second goal is to see if there are other patterns that affect learning outcomes. Perhaps the issue is not funding, but rather the type of school. From examining the type_summary dataframe, we can see that type plays a heavy role, as charter schools have much higher student success rates than district schools. This may also be linked to school size, as charter schools tend to be smaller in terms of student population than district schools. The small to medium size charter schools seem to have much higher overall success rates than the large district schools (90% to 50% success rate disparity).

---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = len(school_data_complete['school_name'].unique())
school_count

15

In [3]:
# Calculate the total number of students
#student_count = school_data_complete['school_name'].value_counts().sum()
#student_count

student_count=len(school_data_complete['Student ID'].unique())
student_count


39170

In [4]:
# Calculate the total budget
total_budget = school_data_complete['budget'].unique().sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = {}
#index=['Total Schools','Total Students','Total Budget','Average Math Score','% Passing Math','% Passing Reading','% Overall Passing']
district_summary["Total Schools"]=school_count
district_summary["Total Students"] =student_count
district_summary["Total Budget"] = total_budget
district_summary["Average Math Score"] = average_math_score
district_summary["% Passing Math"] = passing_math_percentage
district_summary["% Passing Reading"] = passing_reading_percentage
district_summary["% Overall Passing"] = overall_passing_rate
district_summary=pd.DataFrame(district_summary,index=[0])


# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,74.980853,85.805463,65.172326


## School Summary

In [11]:
# Use the code provided to select all of the school types
school_types = school_data_complete.groupby("school_name")['type'].unique().str[0]
school_types

school_name
Bailey High School       District
Cabrera High School       Charter
Figueroa High School     District
Ford High School         District
Griffin High School       Charter
Hernandez High School    District
Holden High School        Charter
Huang High School        District
Johnson High School      District
Pena High School          Charter
Rodriguez High School    District
Shelton High School       Charter
Thomas High School        Charter
Wilson High School        Charter
Wright High School        Charter
Name: type, dtype: object

In [12]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby("school_name")['student_name'].count()
per_school_counts

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [13]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data_complete.groupby("school_name")['budget'].unique().str[0]
per_school_budget


per_school_capita = per_school_budget / per_school_counts
per_school_capita
#total=0
#for item in per_school_budget2:
#    print(item)

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [14]:
# Calculate the average test scores per school
per_school_math = school_data_complete.groupby("school_name")['math_score'].mean()
per_school_math


per_school_reading = school_data_complete.groupby("school_name")['reading_score'].mean()
per_school_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [15]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete['math_score']>70]
school_students_passing_math = students_passing_math.groupby("school_name")['student_name'].count()
school_students_passing_math

school_name
Bailey High School       3216
Cabrera High School      1664
Figueroa High School     1880
Ford High School         1801
Griffin High School      1317
Hernandez High School    3001
Holden High School        387
Huang High School        1847
Johnson High School      3040
Pena High School          882
Rodriguez High School    2562
Shelton High School      1583
Thomas High School       1475
Wilson High School       2076
Wright High School       1625
Name: student_name, dtype: int64

In [16]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[school_data_complete['reading_score']>70]
school_students_passing_reading = students_passing_reading.groupby("school_name")['student_name'].count()
school_students_passing_reading

school_name
Bailey High School       3946
Cabrera High School      1744
Figueroa High School     2313
Ford High School         2123
Griffin High School      1371
Hernandez High School    3624
Holden High School        396
Huang High School        2299
Johnson High School      3727
Pena High School          887
Rodriguez High School    3109
Shelton High School      1631
Thomas High School       1519
Wilson High School       2129
Wright High School       1682
Name: student_name, dtype: int64

In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [18]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
overall_passing_rate

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = {}
per_school_summary['% Passing Math']=per_school_passing_math
per_school_summary['Average Math Score']=per_school_math
per_school_summary['Average Reading Score']=per_school_reading
per_school_summary['% Passing Reading']=per_school_passing_reading
per_school_summary['% Overall Passing']=overall_passing_rate
per_school_summary['Total School Budget']=per_school_budget
per_school_summary["Per Student Budget"]=per_school_capita
per_school_summary["Total Students"]=per_school_counts
per_school_summary["School Type"]=school_types



#df['value'] = df['value'].str[0]
per_school_summary['Total School Budget']=per_school_summary['Total School Budget']
per_school_summary['Per Student Budget']=per_school_summary['Per Student Budget']

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary=pd.DataFrame(per_school_summary)

# Display the DataFrame
per_school_summary

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type
school_name,,,,,,,,,
Bailey High School,64.630225,77.048432,81.033963,79.300643,54.642283,"$3,124,928.00",$628.00,4976,District
Cabrera High School,89.558665,83.061895,83.975780,93.864370,91.334769,"$1,081,356.00",$582.00,1858,Charter
Figueroa High School,63.750424,76.711767,81.158020,78.433367,53.204476,"$1,884,411.00",$639.00,2949,District
Ford High School,65.753925,77.102592,80.746258,77.510040,54.289887,"$1,763,916.00",$644.00,2739,District
Griffin High School,89.713896,83.351499,83.816757,93.392371,90.599455,"$917,500.00",$625.00,1468,Charter
Hernandez High School,64.746494,77.289752,80.934412,78.187702,53.527508,"$3,022,020.00",$652.00,4635,District
Holden High School,90.632319,83.803279,83.814988,92.740047,89.227166,"$248,087.00",$581.00,427,Charter
Huang High School,63.318478,76.629414,81.182722,78.813850,53.513884,"$1,910,635.00",$655.00,2917,District
Johnson High School,63.852132,77.072464,80.966394,78.281874,53.539172,"$3,094,650.00",$650.00,4761,District


## Highest-Performing Schools (by % Overall Passing)

In [20]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values('% Overall Passing',ascending=False)
top_schools.head(5)

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type
school_name,,,,,,,,,
Cabrera High School,89.558665,83.061895,83.975780,93.864370,91.334769,"$1,081,356.00",$582.00,1858,Charter
Thomas High School,90.214067,83.418349,83.848930,92.905199,90.948012,"$1,043,130.00",$638.00,1635,Charter
Griffin High School,89.713896,83.351499,83.816757,93.392371,90.599455,"$917,500.00",$625.00,1468,Charter
Wilson High School,90.932983,83.274201,83.989488,93.254490,90.582567,"$1,319,574.00",$578.00,2283,Charter
Pena High School,91.683992,83.839917,84.044699,92.203742,90.540541,"$585,858.00",$609.00,962,Charter


## Bottom Performing Schools (By % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values('% Overall Passing')
bottom_schools.head(5)

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type
school_name,,,,,,,,,
Rodriguez High School,64.066017,76.842711,80.744686,77.744436,52.988247,"$2,547,363.00",$637.00,3999,District
Figueroa High School,63.750424,76.711767,81.158020,78.433367,53.204476,"$1,884,411.00",$639.00,2949,District
Huang High School,63.318478,76.629414,81.182722,78.813850,53.513884,"$1,910,635.00",$655.00,2917,District
Hernandez High School,64.746494,77.289752,80.934412,78.187702,53.527508,"$3,022,020.00",$652.00,4635,District
Johnson High School,63.852132,77.072464,80.966394,78.281874,53.539172,"$3,094,650.00",$650.00,4761,District


## Math Scores by Grade

In [22]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
ninth_grade_math_scores


tenth_grade_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grade_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean() 
twelfth_grade_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`

math_scores_by_grade = {}
math_scores_by_grade['Ninth Graders']=ninth_grade_math_scores
math_scores_by_grade['Tenth Graders']=tenth_grade_math_scores
math_scores_by_grade['Eleventh Graders']=eleventh_grade_math_scores
math_scores_by_grade['Twelfth Graders']=twelfth_grade_math_scores

math_scores_by_grade=pd.DataFrame(math_scores_by_grade)


# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,Ninth Graders,Tenth Graders,Eleventh Graders,Twelfth Graders
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grade_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grade_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean() 
twelfth_grade_reading_scores =  twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = {}
reading_scores_by_grade['Ninth Graders']=ninth_grade_reading_scores
reading_scores_by_grade['Tenth Graders']=tenth_grade_reading_scores
reading_scores_by_grade['Eleventh Graders']=eleventh_grade_reading_scores
reading_scores_by_grade['Twelfth Graders']=twelfth_grade_reading_scores

reading_scores_by_grade=pd.DataFrame(reading_scores_by_grade)

# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,Ninth Graders,Tenth Graders,Eleventh Graders,Twelfth Graders
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [24]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [25]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
school_spending_df.head()

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type
school_name,,,,,,,,,
Bailey High School,64.630225,77.048432,81.033963,79.300643,54.642283,"$3,124,928.00",$628.00,4976,District
Cabrera High School,89.558665,83.061895,83.975780,93.864370,91.334769,"$1,081,356.00",$582.00,1858,Charter
Figueroa High School,63.750424,76.711767,81.158020,78.433367,53.204476,"$1,884,411.00",$639.00,2949,District
Ford High School,65.753925,77.102592,80.746258,77.510040,54.289887,"$1,763,916.00",$644.00,2739,District
Griffin High School,89.713896,83.351499,83.816757,93.392371,90.599455,"$917,500.00",$625.00,1468,Charter


In [26]:
# Use `pd.cut` to categorize spending based on the bins.
#df1['Avg_Annual'] = df1['Avg_Annual'].str.replace('$', '')
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"].str.replace('$','').astype(float), bins=spending_bins, labels=labels, include_lowest=True)
school_spending_df

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,64.630225,77.048432,81.033963,79.300643,54.642283,"$3,124,928.00",$628.00,4976,District,$585-630
Cabrera High School,89.558665,83.061895,83.975780,93.864370,91.334769,"$1,081,356.00",$582.00,1858,Charter,<$585
Figueroa High School,63.750424,76.711767,81.158020,78.433367,53.204476,"$1,884,411.00",$639.00,2949,District,$630-645
Ford High School,65.753925,77.102592,80.746258,77.510040,54.289887,"$1,763,916.00",$644.00,2739,District,$630-645
Griffin High School,89.713896,83.351499,83.816757,93.392371,90.599455,"$917,500.00",$625.00,1468,Charter,$585-630
Hernandez High School,64.746494,77.289752,80.934412,78.187702,53.527508,"$3,022,020.00",$652.00,4635,District,$645-680
Holden High School,90.632319,83.803279,83.814988,92.740047,89.227166,"$248,087.00",$581.00,427,Charter,<$585
Huang High School,63.318478,76.629414,81.182722,78.813850,53.513884,"$1,910,635.00",$655.00,2917,District,$645-680
Johnson High School,63.852132,77.072464,80.966394,78.281874,53.539172,"$3,094,650.00",$650.00,4761,District,$645-680


In [27]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [28]:
# Assemble into DataFrame
spending_summary = {}

spending_summary['Average Math Score']=spending_math_scores
spending_summary['Average Reading Score']=spending_reading_scores
spending_summary['% Passing Math']=spending_passing_math 
spending_summary['% Passing Reading']=spending_passing_reading
spending_summary['% Overall Passing']=overall_passing_spending 

spending_summary=pd.DataFrame(spending_summary)


# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,90.350436,93.325838,90.369459
$585-630,81.899826,83.155286,83.980055,89.378647,81.418596
$630-645,78.518855,81.624473,70.946108,81.648261,62.857656
$645-680,76.997210,81.027843,63.972368,78.427809,53.526855


## Scores by School Size

In [29]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [30]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

#per_school_summary

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=labels, include_lowest=True)
per_school_summary

,% Passing Math,Average Math Score,Average Reading Score,% Passing Reading,% Overall Passing,Total School Budget,Per Student Budget,Total Students,School Type,School Size
school_name,,,,,,,,,,
Bailey High School,64.630225,77.048432,81.033963,79.300643,54.642283,"$3,124,928.00",$628.00,4976,District,Large (2000-5000)
Cabrera High School,89.558665,83.061895,83.975780,93.864370,91.334769,"$1,081,356.00",$582.00,1858,Charter,Medium (1000-2000)
Figueroa High School,63.750424,76.711767,81.158020,78.433367,53.204476,"$1,884,411.00",$639.00,2949,District,Large (2000-5000)
Ford High School,65.753925,77.102592,80.746258,77.510040,54.289887,"$1,763,916.00",$644.00,2739,District,Large (2000-5000)
Griffin High School,89.713896,83.351499,83.816757,93.392371,90.599455,"$917,500.00",$625.00,1468,Charter,Medium (1000-2000)
Hernandez High School,64.746494,77.289752,80.934412,78.187702,53.527508,"$3,022,020.00",$652.00,4635,District,Large (2000-5000)
Holden High School,90.632319,83.803279,83.814988,92.740047,89.227166,"$248,087.00",$581.00,427,Charter,Small (<1000)
Huang High School,63.318478,76.629414,81.182722,78.813850,53.513884,"$1,910,635.00",$655.00,2917,District,Large (2000-5000)
Johnson High School,63.852132,77.072464,80.966394,78.281874,53.539172,"$3,094,650.00",$650.00,4761,District,Large (2000-5000)


In [31]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [32]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = {}

size_summary["Average Math Score"]=size_math_scores
size_summary["Average Reading Score"]=size_reading_scores
size_summary["% Passing Math"]=size_passing_math
size_summary["% Passing Reading"]=size_passing_reading
size_summary["% Overall Passing"]=size_overall_passing

size_summary=pd.DataFrame(size_summary)


# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,89.883853
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,90.621535
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,58.286003


## Scores by School Type

In [33]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [34]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = {}
type_summary["Average Math Score"]=average_math_score_by_type
type_summary["Average Reading Score"]=average_reading_score_by_type
type_summary["% Passing Math"]=average_percent_passing_math_by_type
type_summary["% Passing Reading"]=average_percent_passing_reading_by_type
type_summary["% Overall Passing"]=average_percent_overall_passing_by_type

type_summary=pd.DataFrame(type_summary)

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,90.432244
District,76.956733,80.966636,64.302528,78.324559,53.672208
